In [44]:
#環境設置
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import requests
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
import time
driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 


In [ ]:
#廢棄了啦
def primo_crawler(org, url, ISBN, driver, wait, tcn):
    def primo_finding(org, tcn): #primo爬資訊的def ；#tcn = thelist_class_name
        sub_df_lst = []
        try:
            back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
        except:
            back = None
        if back != None:
            back.click()

        thelist = driver.find_elements_by_class_name(tcn)
        if tcn == 'md-2-line.md-no-proxy._md': #如果是東吳或銘傳
            end = int(-2)
        else:
            end = ''

        for row in thelist[:end]:
            plist = row.find_elements_by_tag_name("p")
            where = row.find_elements_by_tag_name("h3")
            i = len(where)
            for sth in plist:
                a = sth.find_elements_by_tag_name("span")
                for _ in range(i): 
                    new_row = [uni_name, where[_].text, a[4].text, a[0].text, url]
                    sub_df_lst.append(new_row)
                    break
                break
        return sub_df_lst

    try:
        # 進入《館藏系統》頁面
        driver.get(url)
        # 等待＂進階查詢的按鈕＂直到出現：click
        wait.until(EC.element_to_be_clickable((By.class_name, 'item-title')[0])).click()

    try: #開始爬蟲
        editions = driver.find_elements_by_class_name('item-title') 
        if len(editions) > 1: #如果最外面有兩個版本(默認點進去不會再分版本了啦)(ex.政大 9789861371955)
            pass
        else: #如果最外面只有一個版本，那有可能點進去還有再分
            time.sleep(3)
            try:
                editions.click()
                editions = driver.find_elements_by_class_name('item-title') #這時候是第二層的分版本了！(ex.政大 9789869109321)
            except:
                pass
        for i in range(len(editions)): #有幾個版本就跑幾次，不管哪一層版本都適用
            into = editions[i].click()
            time.sleep(3)
            df_lst += primo_finding(uni_name, tcn)
            try:
                back2 = driver.find_element_by_class_name("md-icon-button.close-button.full-view-navigation.md-button.md-primoExplore-theme.md-ink-ripple").click()
            except:
                back2 = None
    except:
        pass

    return pd.DataFrame(df_lst)


In [66]:
#台大、政大、淡江、東吳、然後銘傳沒有索書碼QQ(要另外進去但我懶🙄)

org = "銘傳大學"
ISBN = str(9789861371955)
url = "https://uco-mcu.primo.exlibrisgroup.com/discovery/search?query=any,contains," + ISBN + "&tab=Everything&search_scope=MyInst_and_CI&vid=886UCO_MCU:886MCU_INST&lang=zh-tw&offset=0"
tcn = "md-2-line.md-no-proxy._md"

def primo_crawler(org, url, ISBN, driver, wait, tcn):
    primo_lst = []
    def primo_finding(org, tcn): #primo爬資訊的def ；#tcn = thelist_class_name
        sub_df_lst = []
        time.sleep(2)
        try:
            back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
        except:
            back = None
        if back != None:
            back.click()

        thelist = driver.find_elements_by_class_name(tcn)
        if tcn == 'md-2-line.md-no-proxy._md': #如果是東吳或銘傳
            thelist = thelist[0:-2]
        else:
            pass

        for row in thelist:
            plist = row.find_elements_by_tag_name("p")
            where = row.find_elements_by_tag_name("h3")
            i = len(where)
            for sth in plist:
                a = sth.find_elements_by_tag_name("span")
                for _ in range(i): 
                    new_row = [org, where[_].text, a[4].text, a[0].text, url]
                    sub_df_lst.append(new_row)
                    break
                break
        return sub_df_lst

    try:
        # 進入《館藏系統》頁面
        driver.get(url)
        # 等待＂進階查詢的按鈕＂直到出現：click
        time.sleep(3)

        try: #開始爬蟲
            editions = driver.find_elements_by_class_name('item-title') 
            if len(editions) > 1: #如果最外面有兩個版本(默認點進去不會再分版本了啦)(ex.政大 9789861371955)，直接交給下面處理
                pass
            else: #如果最外面只有一個版本，那有可能點進去還有再分，先click進去，再分一個版本跟多個版本的狀況
                time.sleep(2)
                editions[0].click()
                time.sleep(5)
                editions = driver.find_elements_by_class_name('item-title') #這時候是第二層的分版本了！(ex.政大 9789869109321)
                
            try: #先找叉叉確定是不是在最裡層了
                back_check = driver.find_element_by_class_name("md-icon-button.close-button.full-view-navigation.md-button.md-primoExplore-theme.md-ink-ripple")
            except:
                back_check = None
            if back_check == None: #多個版本才要再跑迴圈(找不到叉叉代表不在最裡面，可知不是一個版本)
                for i in range(0, len(editions)): #有幾個版本就跑幾次，不管哪一層版本都適用
                    into = editions[i].click()
                    time.sleep(3)
                    primo_lst += primo_finding(org, tcn)
                    try: 
                        back2 = driver.find_element_by_class_name("md-icon-button.close-button.full-view-navigation.md-button.md-primoExplore-theme.md-ink-ripple").click()
                    except:
                        back2 = None

            else: #如果只有一個版本(有叉叉的意思)，那前面已經click過了不能再做
                time.sleep(3)
                primo_lst += primo_finding(org, tcn)
        except:
            pass
    except:
        pass
    return primo_lst

1
2
[['銘傳大學', '桃園館', '', '不在圖書館內', 'https://uco-mcu.primo.exlibrisgroup.com/discovery/search?query=any,contains,9789861371955&tab=Everything&search_scope=MyInst_and_CI&vid=886UCO_MCU:886MCU_INST&lang=zh-tw&offset=0'], ['銘傳大學', '臺北館', '', '不在圖書館內', 'https://uco-mcu.primo.exlibrisgroup.com/discovery/search?query=any,contains,9789861371955&tab=Everything&search_scope=MyInst_and_CI&vid=886UCO_MCU:886MCU_INST&lang=zh-tw&offset=0']]


In [ ]:
# 交大
#  
def primo_greendot_crawler(org, url, ISBN, driver, wait):
    primo_greendot_lst = []
    def primo_greendot_finding(uni_name): #primo爬資訊的def
        sub_df_lst = []
        try:
            time.sleep(1)
            place = driver.find_element_by_class_name('EXLAvailabilityLibraryName')
            status = driver.find_element_by_id('rtaReturnedStatusTOP010054897964801549636893389809149347790')
            num = driver.find_element_by_xpath('//html/body/div[1]/div[3]/div[1]/div[3]/table/tbody/tr/td[4]/div[5]/form/div[2]/div[2]/div/span[2]/cite')
            new_row = [uni_name, place.text, status.text, num.text, url]
            sub_df_lst.append(new_row)
        except:
            print(QQ)
        return sub_df_lst

    driver.get("https://ustcate0.lib.nctu.edu.tw/primo_library/libweb/action/search.do?fn=search&ct=search&initialSearch=true&mode=Basic&tab=ils_article&indx=1&dum=true&srt=rank&vid=NCTU&frbg=&tb=t&vl%28freeText0%29=" + ISBN + "&scp.scps=scope%3A%28NCTU_AV%29%2Cscope%3A%28NCTU_BOOK%29%2Cscope%3A%28NCTU_DT%29%2Cscope%3A%28NCTU_JOURNAL%29%2Cscope%3A%28NCTU%29%2Cprimo_central_multiple_fe&vl%281UI0%29=contains") # 更改網址以前往不同網頁
    try:
        try: #只有一個版本
            time.sleep(2)
            place_click = driver.find_element_by_id('exlidResult0-LocationsTab').click()
            primo_greendot_lst += primo_greendot_finding(org)
        except: #有多個版本，所以要點進去再做
            time.sleep(2)
            manyeditions = driver.find_element_by_class_name('EXLBriefResultsDisplayMultipleLink')    
            for i in range(1, 10): #假設有十個版本吧
                try:
                    editions = driver.find_element_by_xpath('//html/body/div[1]/div[3]/div[1]/div[3]/table/tbody/tr[' + str(i) + ']/td[4]/div[2]/div/ul/li[2]/a').click()
                except:
                    pass
                primo_greendot_lst += primo_greendot_finding(org)
    except:
        print(None)


In [13]:
# 台大
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 
driver.get("https://ntu.primo.exlibrisgroup.com/discovery/search?vid=886NTU_INST:886NTU_INST") # 更改網址以前往不同網頁
ISBN = input()
element = driver.find_element_by_id('searchBar')
element.send_keys(ISBN)
search_gogogo = driver.find_element_by_class_name('zero-margin.button-confirm.large-icon.md-button.md-primoExplore-theme.md-ink-ripple').click()
time.sleep(2)
title = driver.find_element_by_class_name('item-title').click()

time.sleep(3)
try:
    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
except:
    back = None
if back != None:
    back.click()
thelist = driver.find_elements_by_class_name("layout-align-space-between-center.layout-row.flex-100")
for row in thelist:
    plist = row.find_elements_by_tag_name("p")
    where = row.find_elements_by_tag_name("h3")
    i = len(where)
    for sth in plist:
        a = sth.find_elements_by_tag_name("span")
        for _ in range(i): 
            print('台灣大學', where[_].text, a[0].text, end = "\n")
            break
        break
driver.close()

9789869109321
台灣大學 總圖書館 有可用館藏
台灣大學 辜振甫圖書館 不在圖書館內
台灣大學 辜振甫圖書館 有可用館藏
台灣大學 醫學圖書館 不在圖書館內


In [16]:
# 政大
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 
driver.get("https://nccu.primo.exlibrisgroup.com/discovery/search?vid=886NCCU_INST:886NCCU_INST&lang=zh-tw") # 更改網址以前往不同網頁
ISBN = input()
element = driver.find_element_by_id('searchBar')
element.send_keys(ISBN)
search_gogogo = driver.find_element_by_class_name('zero-margin.button-confirm.large-icon.md-button.md-primoExplore-theme.md-ink-ripple').click()
time.sleep(2)
title = driver.find_element_by_class_name('item-title').click()

time.sleep(3)
try:
    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
except:
    back = None
if back != None:
    back.click()
thelist = driver.find_elements_by_class_name("layout-align-space-between-center.layout-row.flex-100")
for row in thelist:
    plist = row.find_elements_by_tag_name("p")
    where = row.find_elements_by_tag_name("h3")
    i = len(where)
    for sth in plist:
        a = sth.find_elements_by_tag_name("span")
        for _ in range(i): 
            print('政治大學', where[_].text, a[0].text, a[2].text, end = "\n")
            break
        break
driver.close()

9789868879348
政治大學 綜圖 目前可獲得 綜圖B1中文
政治大學 達賢館 目前可獲得 3F達賢書房


In [17]:
# 銘傳
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 
driver.get("https://uco-mcu.primo.exlibrisgroup.com/discovery/search?vid=886UCO_MCU:886MCU_INST&lang=zh-tw") # 更改網址以前往不同網頁
ISBN = input()
element = driver.find_element_by_id('searchBar')
element.send_keys(ISBN)
search_gogogo = driver.find_element_by_class_name('zero-margin.button-confirm.large-icon.md-button.md-primoExplore-theme.md-ink-ripple').click()
time.sleep(2)
title = driver.find_element_by_class_name('item-title').click()

time.sleep(3)
try:
    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
except:
    back = None
if back != None:
    back.click()
thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
for row in thelist[:-2]:
    plist = row.find_elements_by_tag_name("p")
    where = row.find_elements_by_tag_name("h3")
    i = len(where)
    for sth in plist:
        a = sth.find_elements_by_tag_name("span")
        for _ in range(1): 
            print('銘傳大學', where[_].text, a[0].text, end = "\n")
            break
        break
driver.close()

9789869109321
銘傳大學 桃園館 有可用館藏
銘傳大學 臺北館 有可用館藏


In [18]:
# 東吳
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 
driver.get("https://uco-scu.primo.exlibrisgroup.com/discovery/search?vid=886UCO_SCU:886SCU_INST&lang=zh-tw") # 更改網址以前往不同網頁
ISBN = input()
element = driver.find_element_by_id('searchBar')
element.send_keys(ISBN)
search_gogogo = driver.find_element_by_class_name('zero-margin.button-confirm.large-icon.md-button.md-primoExplore-theme.md-ink-ripple').click()
time.sleep(2)
title = driver.find_element_by_class_name('item-title').click()

time.sleep(3)
try:
    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
except:
    back = None
if back != None:
    back.click()
thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
for row in thelist[:-2]:
    plist = row.find_elements_by_tag_name("p")
    where = row.find_elements_by_tag_name("h3")
    i = len(where)
    for sth in plist:
        a = sth.find_elements_by_tag_name("span")
        for _ in range(1): 
            print('東吳大學', where[_].text, a[0].text, end = "\n")
            break
        break 
driver.close()

9789869109321
東吳大學 城區分館書庫 目前可獲得
東吳大學 城區分館書庫 目前可獲得
東吳大學 雙溪中正書庫 目前可獲得


In [21]:
# 淡江
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 
driver.get("https://uco-network.primo.exlibrisgroup.com/discovery/search?vid=886UCO_TKU:886TKU_INST&lang=zh-tw") # 更改網址以前往不同網頁
ISBN = input()
element = driver.find_element_by_id('searchBar')
element.send_keys(ISBN)
search_gogogo = driver.find_element_by_class_name('zero-margin.button-confirm.large-icon.md-button.md-primoExplore-theme.md-ink-ripple').click()
time.sleep(2)
title = driver.find_element_by_class_name('item-title').click()

time.sleep(3)
try:
    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
except:
    back = None
if back != None:
    back.click()
thelist = driver.find_elements_by_class_name("neutralized-button.layout-full-width.layout-display-flex.md-button.md-ink-ripple.layout-row")
for row in thelist:
    plist = row.find_elements_by_tag_name("p")
    where = row.find_elements_by_tag_name("h3")
    i = len(where)
    for sth in plist:
        a = sth.find_elements_by_tag_name("span")
        for _ in range(i): 
            print('淡江大學', where[_].text, a[0].text, end = "\n")
            break
driver.close()

9789869109321
淡江大學 總館 可獲得
淡江大學 蘭陽校園圖書館 可獲得


In [33]:
# 政大
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 
driver.get("https://nccu.primo.exlibrisgroup.com/discovery/search?vid=886NCCU_INST:886NCCU_INST&lang=zh-tw") # 更改網址以前往不同網頁
ISBN = input()
element = driver.find_element_by_id('searchBar')
element.send_keys(ISBN)
search_gogogo = driver.find_element_by_class_name('zero-margin.button-confirm.large-icon.md-button.md-primoExplore-theme.md-ink-ripple').click()
time.sleep(2)
title = driver.find_elements_by_class_name('item-title')[0].click()
time.sleep(1)
firstbook = driver.find_elements_by_class_name('item-title')[0].click()
time.sleep(3)
try:
    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
except:
    back = None
if back != None:
    back.click()
thelist = driver.find_elements_by_class_name("layout-align-space-between-center.layout-row.flex-100")
for row in thelist:
    plist = row.find_elements_by_tag_name("p")
    where = row.find_elements_by_tag_name("h3")
    i = len(where)
    for sth in plist:
        a = sth.find_elements_by_tag_name("span")
        for _ in range(i): 
            print('政治大學', where[_].text, a[0].text, a[2].text, end = "\n")
            break
        break
try:
    back2 = driver.find_element_by_class_name("md-icon-button.close-button.full-view-navigation.md-button.md-primoExplore-theme.md-ink-ripple").click()
except:
    back2 = None
secondbook = driver.find_elements_by_class_name('item-title')[1].click()
time.sleep(3)
try:
    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
except:
    back = None
if back != None:
    back.click()
thelist = driver.find_elements_by_class_name("layout-align-space-between-center.layout-row.flex-100")
for row in thelist:
    plist = row.find_elements_by_tag_name("p")
    where = row.find_elements_by_tag_name("h3")
    i = len(where)
    for sth in plist:
        a = sth.find_elements_by_tag_name("span")
        for _ in range(i): 
            print('政治大學', where[_].text, a[0].text, a[2].text, end = "\n")
            break
        break
    

9789869109321
政治大學 綜圖 不在圖書館內 綜圖B1中文
政治大學 達賢館 不在圖書館內 3F達賢書房
政治大學 綜圖 目前可獲得 綜圖B1中文


In [20]:
# 交大
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 
driver.get("https://ustcate0.lib.nctu.edu.tw/primo_library/libweb/action/search.do?mode=Basic&vid=NCTU&vl(freeText0)=&tab=ils_article&") # 更改網址以前往不同網頁
ISBN = input()
element = driver.find_element_by_id('search_field')
element.send_keys(ISBN)
search_gogogo = driver.find_element_by_id('goButton').click()
time.sleep(2)
place_click = driver.find_element_by_id('exlidResult0-LocationsTab').click()
time.sleep(1)
place = driver.find_element_by_class_name('EXLAvailabilityLibraryName')
status = driver.find_element_by_id('rtaReturnedStatusTOP010054897964801549636893389809149347790')
print('交通大學', place.text, status.text, end = "\n")


driver.close()


9789869109321
交通大學 交大圖書總館 有可用館藏


In [13]:
# 桃園
from selenium import webdriver # 先下載 webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\mayda\Downloads\chromedriver") 
driver.get("https://webpac.typl.gov.tw/advanceSearch")
time.sleep(5)
ISBN = input()
element = driver.find_element_by_name('searchInput')
element.send_keys(ISBN)
select = Select(driver.find_element_by_name('searchField'))
select.select_by_value("ISBN")
search_gogogo = driver.find_element_by_xpath('//html/body/div/div[1]/div/div/div/div/div/div[2]/div/div/form/div[1]/div/input[2]').click()
time.sleep(10)

try:
    show_more = driver.find_element_by_link_text("載入更多").click()
except:
    show_more = None
    
time.sleep(5)  
firstbook = driver.find_element_by_xpath('/html/body/div/div[1]/div[2]/div/div/div[2]/div/div[4]/form/div/div[2]/div[2]/div[1]/h2/a').click()
time.sleep(2) 
table = driver.find_element_by_class_name('bookplace_list')
trlist = table.find_elements_by_tag_name('tr')
for row in trlist:
    tdlist = row.find_elements_by_tag_name('td')
    for sth in tdlist:
        print('桃園市立圖書館', tdlist[0].text, tdlist[3].text, end = "\n")
        break
backtolist = driver.find_element_by_link_text('查詢列表頁').click()
time.sleep(5)
secondbook = driver.find_element_by_xpath('/html/body/div/div[1]/div/div/div/div[2]/div/div[4]/form/div/div[3]/div[2]/div[1]/h2/a').click()
time.sleep(2) 
table = driver.find_element_by_class_name('bookplace_list')
trlist = table.find_elements_by_tag_name('tr')
for row in trlist:
    tdlist = row.find_elements_by_tag_name('td')
    for sth in tdlist:
        print('桃園市立圖書館', tdlist[0].text, tdlist[3].text, end = "\n")
        break
backtolist = driver.find_element_by_link_text('查詢列表頁').click()
driver.close()

桃園市立圖書館 楊梅分館/閱覽區 已被外借/2021-08-12
桃園市立圖書館 總館/桃園分區資源中心 書在館
桃園市立圖書館 八德分館/八德 書在館
桃園市立圖書館 觀音分館/閱覽室 預約待取
桃園市立圖書館 蘆竹分館/3樓書庫 書在館
桃園市立圖書館 龍潭分館/1F開架閱覽區 書在館
桃園市立圖書館 埔心分館/一般書庫 書在館
桃園市立圖書館 龍岡分館/書庫 書在館
桃園市立圖書館 龍岡分館/書庫 書在館
桃園市立圖書館 中壢分館/書庫 書在館
桃園市立圖書館 大林分館/閱覽室 書在館


In [18]:
#基隆市
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\mayda\Downloads\chromedriver") 
driver.get("https://webpac.klccab.gov.tw/webpac/search.cfm") # 更改網址以前往不同網頁
ISBN = input()
element = driver.find_element_by_id('ss_keyword')
element.send_keys(ISBN)
search_gogogo = driver.find_element_by_name('searchBtn').click()
time.sleep(2)
table = driver.find_element_by_class_name('list.list_border')
trlist = table.find_elements_by_tag_name('tr')
for row in trlist[1:]:
    tdlist = row.find_elements_by_tag_name('td')
    for sth in tdlist:
        print('基隆市立圖書館', tdlist[1].text, tdlist[3].text, end = "\n")
        break
    
driver.close()

9789869109321
基隆市立圖書館 館藏地 館藏狀態
